In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
data = []
url_visited = []
url_base = 'https://www.enedis.fr/documents'
url = url_base

for num_page in range(500):
    if num_page % 10 == 0:
        print(num_page)
    response = requests.get(url)
    url_visited.append(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    page_content = soup.select(".press-page__field")
    scrapping = []
    scrap_elem = []
    sep_list = [elem == page_content[0] for elem in page_content]

    for i in range(1, len(sep_list)):
        if sep_list[i]:
            scrapping.append(scrap_elem)
            scrap_elem = []
        else:
            scrap_elem.append([elem for elem in page_content[i].contents if elem != '\n'])
    scrapping.append(scrap_elem)

    for pdf in scrapping:
        new_data = [[""]] * 5
        for elem in pdf:
            if type(elem) == list and len(elem):
                if "attrs" in elem[0].__dict__ and 'href' in elem[0].__dict__.get("attrs") and 'aria-label' not in elem[0].__dict__.get("attrs"):
                    new_data[0] = "https://www.enedis.fr" + elem[0].__dict__.get("attrs").get('href')
                elif "attrs" in elem[0].__dict__ and 'datetime' in elem[0].__dict__.get("attrs"):
                    new_data[2] = elem[0].__dict__.get("attrs").get('datetime')
                elif "attrs" in elem[0].__dict__ and 'aria-label' in elem[0].__dict__.get("attrs"):
                    new_data[4] = elem[0].__dict__.get("attrs").get('aria-label')
                elif new_data[1] == [""]:
                    new_data[1] = str(elem[0]).strip().replace(",", ";")
                else:
                    new_data[3] = str(elem[0]).strip().replace(",", ";")
        data.append(new_data)

    next = soup.select(".pager__next")
    if not next:
        break
    url = url_base + next[0].attrs.get("href")

In [ ]:
df_pdf_enedis = pd.DataFrame(data, columns=['url', 'type', "date", "content", "file_info"])

In [97]:
df_pdf_enedis.to_csv("documents_enedis.csv")